In [1]:
import cv2
import numpy as np
import os
import random
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.model_selection import train_test_split

In [2]:
from PIL import Image
from shutil import copyfile
import warnings
warnings.filterwarnings('ignore')
# Define the input and output directories
input_dir = 'dataset'
output_dir = 'C:/Users/vedpr/Downloads/Orientation_Assignment/'
train_dir_horizontal = os.path.join(output_dir, 'train/horizontal')
train_dir_vertical = os.path.join(output_dir, 'train/vertical')
validation_dir_horizontal = os.path.join(output_dir, 'validation/horizontal')
validation_dir_vertical = os.path.join(output_dir, 'validation/vertical')

# Create output directories if they do not exist
os.makedirs(train_dir_horizontal, exist_ok=True)
os.makedirs(train_dir_vertical, exist_ok=True)
os.makedirs(validation_dir_horizontal, exist_ok=True)
os.makedirs(validation_dir_vertical, exist_ok=True)

# Function to mirror the top half of the image
def mirror_top_half(image):
    width, height = image.size
    top_half = image.crop((0, 0, width, height // 2))
    mirrored_top_half = top_half.transpose(Image.FLIP_TOP_BOTTOM)
    new_image = Image.new('RGB', (width, height))
    new_image.paste(mirrored_top_half, (0, 0))
    new_image.paste(top_half, (0, height // 2))
    return new_image

# Function to mirror the bottom half of the image
def mirror_bottom_half(image):
    width, height = image.size
    bottom_half = image.crop((0, height // 2, width, height))
    mirrored_bottom_half = bottom_half.transpose(Image.FLIP_TOP_BOTTOM)
    new_image = Image.new('RGB', (width, height))
    new_image.paste(mirrored_bottom_half, (0, height // 2))
    new_image.paste(bottom_half, (0, 0))
    return new_image

# Function to mirror the left half of the image
def mirror_left_half(image):
    width, height = image.size
    left_half = image.crop((0, 0, width // 2, height))
    mirrored_left_half = left_half.transpose(Image.FLIP_LEFT_RIGHT)
    new_image = Image.new('RGB', (width, height))
    new_image.paste(mirrored_left_half, (width // 2, 0))
    new_image.paste(left_half, (0, 0))
    return new_image

# Function to mirror the right half of the image
def mirror_right_half(image):
    width, height = image.size
    right_half = image.crop((width // 2, 0, width, height))
    mirrored_right_half = right_half.transpose(Image.FLIP_LEFT_RIGHT)
    new_image = Image.new('RGB', (width, height))
    new_image.paste(mirrored_right_half, (0, 0))
    new_image.paste(right_half, (width // 2, 0))
    return new_image

# Function to process each image and create four augmented images
def process_images(input_dir, train_dir_horizontal, train_dir_vertical, validation_dir_horizontal, validation_dir_vertical):
    all_images = [f for f in os.listdir(input_dir) if f.endswith(('.png', '.jpg', '.jpeg'))]
    random.shuffle(all_images)
    
    split_index = int(len(all_images) * 0.8)
    train_images = all_images[:split_index]
    validation_images = all_images[split_index:]
    
    for i, filename in enumerate(all_images):
        image_path = os.path.join(input_dir, filename)
        image = Image.open(image_path)

        # Create the four augmented images
        mirrored_top_image = mirror_top_half(image)
        mirrored_bottom_image = mirror_bottom_half(image)
        mirrored_left_image = mirror_left_half(image)
        mirrored_right_image = mirror_right_half(image)

        # Determine if the image should go to train or validation set
        if filename in train_images:
            base_dir_horizontal = train_dir_horizontal
            base_dir_vertical = train_dir_vertical
        else:
            base_dir_horizontal = validation_dir_horizontal
            base_dir_vertical = validation_dir_vertical

        # Save the augmented images in respective directories
        base_filename = os.path.splitext(filename)[0]
        mirrored_top_image.save(os.path.join(base_dir_horizontal, f'{base_filename}_mirrored_top.png'))
        mirrored_bottom_image.save(os.path.join(base_dir_horizontal, f'{base_filename}_mirrored_bottom.png'))
        mirrored_left_image.save(os.path.join(base_dir_vertical, f'{base_filename}_mirrored_left.png'))
        mirrored_right_image.save(os.path.join(base_dir_vertical, f'{base_filename}_mirrored_right.png'))

    print("All augmented images have been created and saved.")

# Run the function
process_images(input_dir, train_dir_horizontal, train_dir_vertical, validation_dir_horizontal, validation_dir_vertical)


All augmented images have been created and saved.


In [3]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define directories
base_dir ='C:/Users/vedpr/Downloads/Orientation_Assignment/'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

# Data augmentation and normalization for training
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Data normalization for validation
validation_datagen = ImageDataGenerator(rescale=1.0/255.0)

# Create generators
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

# Build the model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=20,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size
)

# Save the model
model.save('augmented_image_classifier.h5')

# Evaluate the model
loss, accuracy = model.evaluate(validation_generator)
print(f'Validation accuracy: {accuracy * 100:.2f}%')


Found 320 images belonging to 2 classes.
Found 80 images belonging to 2 classes.
Epoch 1/20
10/10 [==============================] - 13s 1s/step - loss: 0.7049 - accuracy: 0.5250 - val_loss: 0.6915 - val_accuracy: 0.5156
Epoch 2/20
10/10 [==============================] - 8s 785ms/step - loss: 0.6947 - accuracy: 0.4969 - val_loss: 0.6900 - val_accuracy: 0.6406
Epoch 3/20
10/10 [==============================] - 8s 775ms/step - loss: 0.6921 - accuracy: 0.5063 - val_loss: 0.6858 - val_accuracy: 0.7656
Epoch 4/20
10/10 [==============================] - 8s 797ms/step - loss: 0.6711 - accuracy: 0.5969 - val_loss: 0.6200 - val_accuracy: 0.6562
Epoch 5/20
10/10 [==============================] - 8s 783ms/step - loss: 0.6205 - accuracy: 0.6562 - val_loss: 0.4659 - val_accuracy: 0.7812
Epoch 6/20
10/10 [==============================] - 8s 824ms/step - loss: 0.5990 - accuracy: 0.6781 - val_loss: 0.4389 - val_accuracy: 0.7812
Epoch 7/20
10/10 [==============================] - 7s 673ms/step - l

In [6]:
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

def classify_image(image_path, model_path='augmented_image_classifier.h5'):
    # Load the trained model
    model = load_model(model_path)
    
    # Load and preprocess the image
    img = image.load_img(image_path, target_size=(150, 150))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0
    
    # Predict the class probabilities
    class_probabilities = model.predict(img_array)[0]
    
    # Determine the predicted class based on probabilities
    if class_probabilities[0] <0.5:
        return 'horizontal'
    else:
        return 'vertical'

# Example usage
image_path = r'C:\Users\vedpr\Downloads\Orientation_Assignment\validation\vertical\airplane_0005_mirrored_left.png'
predicted_class = classify_image(image_path)
print(f"The image is classified as: {predicted_class}")


1/1 [==============================] - 0s 96ms/step
The image is classified as: vertical


In [7]:
# Example usage
image_path = r'C:\Users\vedpr\Downloads\Orientation_Assignment\validation\horizontal\airplane_0005_mirrored_bottom.png'
predicted_class = classify_image(image_path)
print(f"The image is classified as: {predicted_class}")

1/1 [==============================] - 0s 78ms/step
The image is classified as: horizontal
